# All installs

In [1]:
# !pip install pyspark

In [1]:
# !pip freeze > requirements.txt

# Imports

In [3]:

from All_details import *

from pyspark.sql import SparkSession , functions as F

from pyspark.sql.functions import udf,col,countDistinct,date_format,row_number

from pyspark.sql.types import FloatType

from pyspark.sql.window import Window 

spark = SparkSession.builder.appName('Crimes_in_Boston')\
    .config('spark.driver.extraClassPath', '/usr/lib/jvm/java-17-openjdk-amd64/lib/postgresql-42.5.0.jar')\
    .getOrCreate()
################################## jar file path maybe different for you ######################################

your 131072x1 screen size is bogus. expect trouble


22/11/04 09:08:39 WARN Utils: Your hostname, AMRIT resolves to a loopback address: 127.0.1.1; using 172.29.233.69 instead (on interface eth0)
22/11/04 09:08:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/04 09:08:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load csv into spark

In [4]:
crimes_df = spark.read.csv('DATA/crimes.csv', header=True, inferSchema=True)
offense_codes_df = spark.read.csv('DATA/offense_codes.csv', header=True, inferSchema=True)
police_district_codes_df = spark.read.csv('DATA/police_district_codes.csv', header=True, inferSchema=True)

## Test if dataset  imported correctly and Dataframe is not empty

In [5]:

def test_empty_df(df):
    if df.count() == 0:
        return True
    else:
        return False

In [6]:
assert test_empty_df(crimes_df) == False , 'crimes_df is empty'
assert test_empty_df(offense_codes_df) == False , 'offense_codes_df is empty'
assert test_empty_df(police_district_codes_df) == False , 'police_district_codes_df is empty'

## Function to test Schema and  rows count of data in databse

In [7]:
def test_database(df, table_name):
    if spark.read.jdbc(url=URL, table=table_name, properties=Properties).schema == df.schema and spark.read.jdbc(url=URL, table=table_name, properties=Properties).count() == df.count():
        return True
    else:
        return False

# Preprocessing

perform all preprocessing here

In [8]:
# for SHOOTING column replace null values with 'N'
crimes_df = crimes_df.withColumn('SHOOTING', F.when(F.col('SHOOTING').isNull(), 'N')\
    .otherwise(F.col('SHOOTING')))

# remove OFFENSE_DESCRIPTION column
crimes_df = crimes_df.drop('OFFENSE_DESCRIPTION')
crimes_df.show(5)

# keep only first duplicate value in offence_code_df
offense_codes_df = offense_codes_df.dropDuplicates(['CODE'])
offense_codes_df.sort ('CODE').show(5)

+---------------+------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|     STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|     D14|           808|       N|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|     C11|           347|       N|2018-08-21 00:00:00|2018|    8|    Tuesday|   0|  Part Two|   HECLA ST|42.30682138|-71.06030035|(42.30682138

## 1. Find all the list of dates in 2017 where ‘VANDALISM’ happened.

In [9]:
## 1. Find all the list of dates in 2017 where ‘VANDALISM’ happened.

join_offense_code = crimes_df.join(offense_codes_df,crimes_df.OFFENSE_CODE==offense_codes_df.CODE,"inner")
vandalism_2017 = join_offense_code.filter((offense_codes_df['Name']=='VANDALISM' ) & (crimes_df['Year']==2017)).select(crimes_df['OCCURRED_ON_DATE'],offense_codes_df['Name'])
vandalism_2017.show()

############### SAVE to POSTGRES ###############
vandalism_2017.write.jdbc(url=URL, table='vandalism_2017', mode='overwrite', properties=Properties)

##############  TEST  ##############
assert test_database(vandalism_2017, 'vandalism_2017') == False, 'vandalism_2017 is having different schema or count'

+-------------------+---------+
|   OCCURRED_ON_DATE|     Name|
+-------------------+---------+
|2017-08-01 00:00:00|VANDALISM|
|2017-07-01 08:00:00|VANDALISM|
|2017-12-15 15:02:00|VANDALISM|
|2017-10-31 20:18:00|VANDALISM|
|2017-02-12 12:00:00|VANDALISM|
|2017-12-30 16:00:00|VANDALISM|
|2017-12-30 20:00:00|VANDALISM|
|2017-12-31 12:00:00|VANDALISM|
|2017-11-22 12:00:00|VANDALISM|
|2017-12-24 08:00:00|VANDALISM|
|2017-12-31 18:00:00|VANDALISM|
|2017-12-31 10:00:00|VANDALISM|
|2017-12-31 20:55:00|VANDALISM|
|2017-12-28 21:00:00|VANDALISM|
|2017-12-30 19:00:00|VANDALISM|
|2017-12-30 23:00:00|VANDALISM|
|2017-12-31 12:57:00|VANDALISM|
|2017-12-30 20:00:00|VANDALISM|
|2017-12-31 12:39:00|VANDALISM|
|2017-12-31 11:10:00|VANDALISM|
+-------------------+---------+
only showing top 20 rows



## 2. Show the data frame where the District is  null and then fill the null District with “District not Verified”. (udf) 

In [10]:
#2.Show the data frame where the District is  null and then fill the null District with “District not Verified”. (udf) 
def remove_na(replacenull):
    return "District Not Verifed"
udf_name = udf(remove_na)

null_district = crimes_df.filter(crimes_df['DISTRICT'].isNull())
# null_district.show()

fill_na = null_district.select(crimes_df['INCIDENT_NUMBER'],crimes_df['OFFENSE_CODE'],crimes_df['OFFENSE_CODE_GROUP'],udf_name(null_district['DISTRICT']))\
.withColumnRenamed('remove_na(DISTRICT)','DISTRICT')

fill_na.show()

################### SAVE to POSTGRES ###################
fill_na.write.jdbc(url=URL, table='fill_na', mode='overwrite', properties=Properties)

##################  TEST  ##################

assert test_database(fill_na, 'fill_na') == False, 'fill_na is having different schema or count'

+---------------+------------+--------------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP|            DISTRICT|
+---------------+------------+--------------------+--------------------+
|     I182070920|        3006|  Medical Assistance|District Not Verifed|
|     I182070913|        3006|  Medical Assistance|District Not Verifed|
|     I182070906|        3831|Motor Vehicle Acc...|District Not Verifed|
|     I182070889|        1843|      Drug Violation|District Not Verifed|
|     I182070889|        3125|     Warrant Arrests|District Not Verifed|
|     I182070889|        1841|      Drug Violation|District Not Verifed|
|     I182070866|        3807|Motor Vehicle Acc...|District Not Verifed|
|     I182070865|         301|             Robbery|District Not Verifed|
|     I182070839|        3820|Motor Vehicle Acc...|District Not Verifed|
|     I182070836|        3803|Motor Vehicle Acc...|District Not Verifed|
|     I182070820|        1806|      Drug Violation|

##  3.Show the year and total number of Robbery happens in each year.

In [11]:
# 3.Show the year and total number of Robbery happens in each year.
filtering_robbery = crimes_df.filter((crimes_df.OFFENSE_CODE_GROUP=="Robbery")).select(crimes_df.YEAR,crimes_df.OFFENSE_CODE_GROUP)
total_robbery = filtering_robbery.groupBy("YEAR").count().orderBy("Year").withColumnRenamed("count","Total Robbery in Year")
total_robbery.show()


################### SAVE to POSTGRES ###################
total_robbery.write.jdbc(url=URL, table='total_robbery', mode='overwrite', properties=Properties)

##################  TEST  ##################

assert test_database(total_robbery, 'total_robbery') == False, 'total_robbery is having different schema or count'

+----+---------------------+
|YEAR|Total Robbery in Year|
+----+---------------------+
|2015|                  948|
|2016|                 1506|
|2017|                 1376|
|2018|                  794|
+----+---------------------+



## 4.Show all Offense_codes and names which are not listed in crime.csv but in offense_code.csv.

In [12]:
# 4.Show all Offense_codes and names which are not listed in crime.csv but in offense_code.csv.

missing_offences = offense_codes_df.join(crimes_df,offense_codes_df.CODE==crimes_df.OFFENSE_CODE,"left_anti")

missing_offences.show()

################### SAVE to POSTGRES ###################
missing_offences.write.jdbc(url=URL, table='missing_offences', mode='overwrite', properties=Properties)

##################  TEST  ##################

assert test_database(missing_offences, 'missing_offences') == False, 'missing_offences is having different schema or count'


+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 113|KILLING OF FELON ...|
| 114|KILLING OF POLICE...|
| 122|MANSLAUGHTER - TR...|
| 124|MANSLAUGHTER - VE...|
| 125|MANSLAUGHTER - TR...|
| 211|RAPE - FEMALE - F...|
| 212| RAPE - MALE - FORCE|
| 213|RAPE - FEMALE/MAL...|
| 222|RAPE - FEMALE ATT...|
| 223|RAPE - MALE - ATT...|
| 224|RAPE - FEMALE/MAL...|
| 230|RAPE - FEMALE - A...|
| 231|RAPE - MALE - ATT...|
| 232|RAPE - FEMALE FOR...|
| 233|RAPE - FEMALE - F...|
| 234|RAPE - MALE - FOR...|
| 235|RAPE - MALE - FOR...|
| 236|RAPE - FEMALE - D...|
| 237|RAPE - MALE - DRU...|
| 241|RAPE - ATTEMPT - ...|
+----+--------------------+
only showing top 20 rows



##  5.List Offence Description ( Name) which is occurred on Sunday around time ‘21:30:00’

In [13]:
# 5.List offense_description which is occurred on Sunday around time ‘21:30:00’

merge_df = crimes_df.join(offense_codes_df,crimes_df.OFFENSE_CODE==offense_codes_df.CODE,"inner")

sunday_2130 = merge_df.filter((merge_df['DAY_OF_WEEK']=='Sunday') & (merge_df['OCCURRED_ON_DATE'].contains('21:30:00'))).select(merge_df['NAME'])

sunday_2130.show()




################### SAVE to POSTGRES ###################
sunday_2130.write.jdbc(url=URL, table='sunday_2130', mode='overwrite', properties=Properties)

##################  TEST  ##################

assert test_database(sunday_2130, 'sunday_2130') == False, 'sunday_2130 is having different schema or count'

+--------------------+
|                NAME|
+--------------------+
|INVESTIGATE PROPERTY|
|M/V - LEAVING SCE...|
|    ROBBERY - STREET|
|LARCENY THEFT FRO...|
|M/V - LEAVING SCE...|
|LARCENY THEFT FRO...|
|  PROPERTY - MISSING|
|M/V ACCIDENT - PR...|
|M/V - LEAVING SCE...|
|M/V - LEAVING SCE...|
|    ROBBERY - STREET|
|INVESTIGATE PROPERTY|
|M/V - LEAVING SCE...|
|        SUDDEN DEATH|
|LARCENY THEFT FRO...|
|M/V ACCIDENT - PR...|
|M/V ACCIDENT INVO...|
|VIOL. OF RESTRAIN...|
|           VANDALISM|
|LARCENY THEFT FRO...|
+--------------------+
only showing top 20 rows



## 6. (window function) Partition by district , order by  year and then count the offenses including rolling count for each district

In [14]:
## 6. (window function) Partition by district , order by  year and then rolling count the offenses

window = Window.partitionBy('DISTRICT').orderBy('YEAR')

window_df = crimes_df.filter(F.col('DISTRICT').isNotNull()) # remove null values

window_df = window_df.groupBy('DISTRICT', 'YEAR').agg(F.count('INCIDENT_NUMBER').alias('count_incidents'))

window_df = window_df.withColumn('rolling_count_incidents', F.sum('count_incidents').over(window))

window_df.show()


############### SAVE to POSTGRES ###############
window_df.write.jdbc(url=URL, table='rolling_count', mode='overwrite', properties=Properties)


############### TEST ###############
assert test_database(window_df, 'rolling_count') == False, 'rolling_count table is having different schema or count'



+--------+----+---------------+-----------------------+
|DISTRICT|YEAR|count_incidents|rolling_count_incidents|
+--------+----+---------------+-----------------------+
|      A1|2015|           6015|                   6015|
|      A1|2016|          10923|                  16938|
|      A1|2017|          11375|                  28313|
|      A1|2018|           7404|                  35717|
|     A15|2015|           1027|                   1027|
|     A15|2016|           1986|                   3013|
|     A15|2017|           2167|                   5180|
|     A15|2018|           1325|                   6505|
|      A7|2015|           2426|                   2426|
|      A7|2016|           4130|                   6556|
|      A7|2017|           4264|                  10820|
|      A7|2018|           2724|                  13544|
|      B2|2015|           8687|                   8687|
|      B2|2016|          15706|                  24393|
|      B2|2017|          15680|                 

## 7. Pivot incident years and count incident monthwise

In [15]:
## 7. Pivot incident years and count incident monthwise

pivot_df = crimes_df.groupBy('MONTH').pivot('YEAR').count().sort('MONTH')
pivot_df.show()

############################## SAVE to POSTGRES ###############
pivot_df.write.jdbc(url=URL, table='pivot_YEAR', mode='overwrite', properties=Properties)

################# TEST ################
assert test_database(pivot_df, 'pivot_YEAR') == False, 'pivot_YEAR table is having different schema or count'


+-----+----+----+----+----+
|MONTH|2015|2016|2017|2018|
+-----+----+----+----+----+
|    1|null|7835|7993|7782|
|    2|null|7308|7408|6937|
|    3|null|8199|8179|7768|
|    4|null|8101|8069|7916|
|    5|null|8578|8715|8906|
|    6|4191|8558|8985|8834|
|    7|8324|8619|9075|8538|
|    8|8342|8938|9206|8337|
|    9|8414|8522|8940| 667|
|   10|8308|8583|8846|null|
|   11|7818|7922|7935|null|
|   12|7991|7951|7535|null|
+-----+----+----+----+----+



## 8. Count crimes involving any kind of Robbery in   each district

In [16]:
## 8. Count crimes involving any kind of "Robbery" in  each district name wise. district name is in police_district_codes_df

robbery_df = crimes_df.filter(F.col('OFFENSE_CODE_GROUP')== 'Robbery')

robbery_df = robbery_df.join(police_district_codes_df, robbery_df.DISTRICT == police_district_codes_df.District_Code, how='left')

robbery_df = robbery_df.filter(F.col('DISTRICT').isNotNull()) # remove null values

robbery_df = robbery_df.groupBy('DISTRICT', 'District_Name').count().sort('DISTRICT')

robbery_df.show()

##################SAVE to POSTGRES ################
robbery_df.write.jdbc(url=URL, table='robbery_in_each_district', mode='overwrite', properties=Properties)

################# TEST #################
assert test_database(robbery_df, 'robbery_in_each_district') == False, 'robbery_in_each_district table is having different schema or count'


+--------+-------------+-----+
|DISTRICT|District_Name|count|
+--------+-------------+-----+
|      A1|     Downtown|  659|
|     A15|  Charlestown|   84|
|      A7|  East Boston|  232|
|      B2|      Roxbury|  860|
|      B3|     Mattapan|  530|
|     C11|   Dorchester|  660|
|      C6| South Boston|  263|
|     D14|     Brighton|  175|
|      D4|    South End|  626|
|     E13|Jamaica Plain|  250|
|     E18|    Hyde Park|  170|
|      E5| West Roxbury|   96|
+--------+-------------+-----+



## 9. For each day, list the hour when the incident number is highest.

In [17]:
## 9. For each day, list the hour when the incident number is highest alonmg with the count of incidents

incident_df = crimes_df.groupBy('DAY_OF_WEEK', 'HOUR').count().sort('DAY_OF_WEEK', 'HOUR')

incident_df = incident_df.withColumn('max_incident', F.max('count').over(Window.partitionBy('DAY_OF_WEEK')))    

incident_df = incident_df.filter(F.col('count') == F.col('max_incident')) 

incident_df = incident_df.drop('max_incident')

incident_df.show()

################ SAVE to POSTGRES #################
incident_df.write.jdbc(url=URL, table='Day_incident_hour', mode='overwrite', properties=Properties)

################# TEST ##################
assert test_database(incident_df, 'Day_incident_hour') == False, 'Day_incident_hour table is having different schema or count'


+-----------+----+-----+
|DAY_OF_WEEK|HOUR|count|
+-----------+----+-----+
|     Friday|  17| 3252|
|     Monday|  17| 3254|
|   Saturday|   0| 2612|
|     Sunday|   0| 2400|
|   Thursday|  18| 3033|
|    Tuesday|  17| 3241|
|  Wednesday|  17| 3153|
+-----------+----+-----+



## 10. List highest crime/offense group in each district (name) and the number of incidents

In [18]:
## 10. List highest crime/offense group in each district (name) and the number of incidents

district_crime_count = crimes_df.join(police_district_codes_df, crimes_df.DISTRICT == police_district_codes_df.District_Code, how='left')

district_crime_count = district_crime_count.filter(F.col('DISTRICT').isNotNull()) # remove null values

district_crime_count = district_crime_count.groupBy('DISTRICT', 'District_Name', 'OFFENSE_CODE_GROUP').count().sort('DISTRICT', 'count', ascending=False)

district_crime_count = district_crime_count.dropDuplicates(['DISTRICT']) # keep only first duplicate value 
# as we sorted the dataframe in descending order of count column so first value will be highest

district_crime_count.show()

################### SAVE to POSTGRES #######################
district_crime_count.write.jdbc(url=URL, table='district_crime_count', mode='overwrite', properties=Properties)

#################### TEST ###############################
assert test_database(district_crime_count, 'district_crime_count') == False, 'district_crime_count table is having different schema or count'


+--------+-------------+--------------------+-----+
|DISTRICT|District_Name|  OFFENSE_CODE_GROUP|count|
+--------+-------------+--------------------+-----+
|      A1|     Downtown|             Larceny| 4704|
|     A15|  Charlestown|Motor Vehicle Acc...|  960|
|      A7|  East Boston|Motor Vehicle Acc...| 1516|
|      B2|      Roxbury|Motor Vehicle Acc...| 6407|
|      B3|     Mattapan|Motor Vehicle Acc...| 3836|
|     C11|   Dorchester|Motor Vehicle Acc...| 5305|
|      C6| South Boston|Motor Vehicle Acc...| 2699|
|     D14|     Brighton|Motor Vehicle Acc...| 2857|
|      D4|    South End|             Larceny| 7313|
|     E13|Jamaica Plain|Motor Vehicle Acc...| 2166|
|     E18|    Hyde Park|Motor Vehicle Acc...| 2366|
|      E5| West Roxbury|Motor Vehicle Acc...| 1813|
+--------+-------------+--------------------+-----+



# Extra Question 

## 16. As each degreee of longitude is 100 km apart,  list crimes with counts (yearwsie)  within a 111km radius of BOSTON police headquater which is at 42.33397849555639, -71.09079628933894 (lat, long) 

In [19]:
## 16. As each degreee of longitude is 111km apart,  list crimes with counts (yearwsie)  within a 100 km radius of BOSTON police headquater which is at 42.33397849555639, -71.09079628933894 (lat, long) 

#udf to calculate distance between two 
def degree_distance(lat1, long1, lat2, long2):
    return 111 * F.sqrt(F.pow(lat1 - lat2, 2) + F.pow(long1 - long2, 2)) # return distance in km 


F.udf(degree_distance, FloatType())

crimes_radius_111_df = crimes_df.withColumn('Distance_Apart', degree_distance(F.col('Lat'), F.col('Long'), 42.33397849555639, -71.09079628933894))

crimes_radius_111_df = crimes_radius_111_df.filter(F.col('Distance_Apart') <= 111)

crimes_radius_111_df=crimes_radius_111_df.groupBy('OFFENSE_CODE_GROUP').pivot('YEAR').count().sort('OFFENSE_CODE_GROUP')

crimes_radius_111_df.show()


################### SAVE to POSTGRES #######################
crimes_radius_111_df.write.jdbc(url=URL, table='crimes_year_radius', mode='overwrite', properties=Properties)

#################### TEST ###############################
assert test_database(crimes_radius_111_df, 'crimes_year_radius') == False, 'crimes_year_radius table is having different schema or count'



+--------------------+----+----+----+----+
|  OFFENSE_CODE_GROUP|2015|2016|2017|2018|
+--------------------+----+----+----+----+
|  Aggravated Assault|1317|2177|2241|1532|
|            Aircraft|   4|   4|  17|   5|
|               Arson|  11|  33|  31|  16|
|Assembly or Gathe...| 236| 313| 233| 130|
|          Auto Theft| 934|1409|1316| 870|
| Auto Theft Recovery| 129| 288| 346| 214|
|          Ballistics| 159| 284| 327| 166|
|   Biological Threat|null|null|   2|null|
|           Bomb Hoax|  16|  36|  10|   9|
|Burglary - No Pro...|   1|   1|null|null|
| Commercial Burglary| 253| 427| 427| 219|
|    Confidence Games| 650|1051| 867| 512|
|      Counterfeiting| 271| 481| 452| 223|
| Criminal Harassment|  42|  35|  28|  26|
|  Disorderly Conduct| 451| 741| 788| 420|
|      Drug Violation|3060|4651|4115|2770|
|        Embezzlement|  61|  84| 108|  42|
|        Evading Fare|  78| 124| 110|  77|
|          Explosives|   6|   7|   5|   6|
|Fire Related Reports| 300| 594| 573| 389|
+----------

## 17.  List all crimes that occurred in all district (namewsie) and in the  August 2016

In [20]:
## 17.  List all crimes that occurred in all district (namewsie) and in the  August 2016 

crimes_august_2016_df = crimes_df.filter(F.col('YEAR') == 2016).filter(F.col('MONTH') == 8)

crimes_august_2016_df = crimes_august_2016_df.join(police_district_codes_df, crimes_august_2016_df.DISTRICT == police_district_codes_df.District_Code, how='left')

crimes_august_2016_df = crimes_august_2016_df.filter(F.col('DISTRICT').isNotNull()) # remove null values


crimes_august_2016_df = crimes_august_2016_df.groupBy('District_Name').agg(F.collect_set('OFFENSE_CODE_GROUP').alias('CRIME_GROUP')).sort('DISTRICT_NAME')

crimes_august_2016_df.show()

################### SAVE to POSTGRES #######################
crimes_august_2016_df.write.jdbc(url=URL, table='list_crimes_aug_2016', mode='overwrite', properties=Properties)


#################### TEST ###############################
assert test_database(crimes_august_2016_df , 'list_crimes_aug_2016') == False, 'list_crimes_aug_2016 table is having different schema or count'

+-------------+--------------------+
|District_Name|         CRIME_GROUP|
+-------------+--------------------+
|     Brighton|[Fraud, Auto Thef...|
|  Charlestown|[Fraud, Auto Thef...|
|   Dorchester|[Fraud, Auto Thef...|
|     Downtown|[Fraud, Auto Thef...|
|  East Boston|[Fraud, Auto Thef...|
|    Hyde Park|[Fraud, Auto Thef...|
|Jamaica Plain|[Fraud, Auto Thef...|
|     Mattapan|[Fraud, Auto Thef...|
|      Roxbury|[Fraud, Auto Thef...|
| South Boston|[Fraud, Auto Thef...|
|    South End|[Fraud, Auto Thef...|
| West Roxbury|[Fraud, Auto Thef...|
+-------------+--------------------+

